    from IPython import get_ipython; ip = get_ipython()
    %reload_ext pidgin.tangle

`import pidgin.tangle` modifies the `get_ipython().input_transformer_manager` to accept __Markdown__ source.  The `pidgin.tangle` module
exports:
    
* `pidgin.tangle.markdown_to_python` - is a semi-lossless __Markdown__ to __Python__ converter.


In [1]:
    import doctest, re, ast, mistune, textwrap, functools, itertools, IPython, importnb, fnmatch
    __all__ = 'markdown_to_python',

`quote` wrotes non code objects in triple ticks.

In [2]:
    def quote(str, punc=''):
        str, leading_ws = ''.join(str), []
        lines = str.splitlines(True)
        _ = '"""'
        if _ in str: _ = "'''"
        if not str.strip(): _ = punc = ''
        while lines and (not lines[0].strip()): leading_ws.append(lines.pop(0))    
        str = ''.join(lines)
        end = len(str.rstrip())
        str, ending_ws = str[:end], str[end:]
        if str and str.endswith(_[0]): str += ' '                    
        return F"{''.join(leading_ws)}{_}{str}{_}{punc}{ending_ws}"

`get_first_line` get the first non-`iter`able strings in `lines`

In [3]:
    def get_first_line(lines, line=''):
        for line in lines or ['']: 
            if line.strip(): break
        return line

`get_line_indent` computes the indent of a string.

In [4]:
    def get_line_indent(line):  return len(line) - len(line.lstrip())

The __Lexer__ s only consider coarse features of the markdown spec.  

In [5]:
    def _has_return(code):
        code = '\n'.join(code)
        if 'return ' not in code: return False
        code = importnb.loader.dedent(code)
        try:
            node = ast.parse(code)
            while hasattr(node, 'body'): node = node.body[-1]
            return isinstance(node, ast.Return)
        except: ...  

In [ ]:
    def hanging_indent(str, indent):
        out = """"""
        for line in str.splitlines(True):
            if not line.strip(): 
                out += line
            else:
                if out.strip(): out += line
                else: out += indent+line
        return out

## Exports

In [ ]:
    class PidginBlockGrammar(mistune.BlockGrammar):
        doctest = doctest.DocTestParser._EXAMPLE_RE
        block_html = re.compile(
            r'^ *(?:%s|%s|%s) *(?:\n{2,}|\s*$)' % (
                r'<!--[\s\S]*?-->|<!DOCTYPE [\s\S]*?>|<\?[\s\S]*?\?>',
                r'<(%s)((?:%s)*?)>([\s\S]*?)<\/\1>' % (mistune._block_tag, mistune._valid_attr),
                r'<%s(?:%s)*?\s*\/?>' % (mistune._block_tag, mistune._valid_attr)))       


In [ ]:
    class PidginBlockLexer(mistune.BlockLexer): 
        grammar_class = PidginBlockGrammar        
        @staticmethod
        def to_string(m): return m.string[slice(*m.span())].splitlines(True)

        def parse_doctest(self, m):
            self.tokens.append({'type': 'doctest', 'text': m.string[slice(*m.span())]})
        
    PidginBlockLexer.default_rules = list(PidginBlockLexer.default_rules)
    PidginBlockLexer.default_rules.insert(
        PidginBlockLexer.default_rules.index('block_quote'), 'doctest'
    )
    PidginBlockLexer.footnote_rules = list(PidginBlockLexer.footnote_rules)
    PidginBlockLexer.footnote_rules.insert(
        PidginBlockLexer.footnote_rules.index('block_quote'), 'doctest'
    )
    PidginBlockLexer.list_rules = list(PidginBlockLexer.list_rules)
    PidginBlockLexer.list_rules.insert(
        PidginBlockLexer.list_rules.index('block_quote'), 'doctest'
    )

In [ ]:
    class TanglePidginBlockLexer(PidginBlockLexer):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            self.original = None
            self.raw, self.unindented, self.indented = [], [], []
            self.min_indent = 0
            
        def parse(self, text, rules=None): 
            
            if any((self.original, self.raw, self.unindented, self.indented)): 
                return super().parse(text, rules)
            
            self.original = ''.join(text).splitlines(True)
            tokens = super().parse(text, rules)
            
            while self.original:  self.raw.append(self.original.pop(0))
            self.min_indent = self.min_indent or 4
            self.format(punc=';')
            self.indent()
            final = ''.join(self.indented)
            return final
            
        def clear_original(self):
            while self.original and not self.original[0].strip(): 
                self.raw += [self.original.pop(0)]
                
        def pop_text(self, m)->str:
            text = []
            lines = self.to_string(m)
            # Remove empty lines
            while lines and not lines[0].strip(): lines.pop(0)
            
            # Drop the lines from the original body
            if self.original:
                while lines:
                    line = lines.pop(0)
                    if line.strip():
                        while line.strip() not in self.original[0]: 
                            text += [self.original.pop(0)]
                        text += [self.original.pop(0)]
            return text
        
        def parse_generic(self, m): 
            self.raw.extend(self.pop_text(m))
            
        parse_doctest = parse_block_html = parse_block_quote = parse_fences =\
        parse_heading = parse_hrule = parse_lheading = parse_newline =\
        parse_nptable = parse_paragraph = parse_table = parse_text =  parse_generic
        
        def parse_block_code(self, m=None):
            # The body goes about the code, the buffer is non-code.
            self.format()
            self.raw.extend(m and self.pop_text(m) or [])
            m and super().parse_block_code(m)
            self.indent()
            
        def format(self, punc=''):
            self.clear_original()            
            if self.raw: 
                last_line = get_first_line(reversed(self.indented)).rstrip()
                lines = [line for line in self.raw]
                #                 if not last_line.endswith(('"""', "'''")):
                #                     # Allow authors to provide literal quotes
                lines = [quote(lines, punc)]
                self.unindented.extend(lines)
                self.raw = []
                        
            
        def indent(self):
            # Extract the first line of the current code block.
            first_line = get_first_line(self.raw)
            # Construct the code we'll 
            code = ''.join(self.raw)                            
            body = ''.join(self.unindented)            
            
            # The previous last line append
            last_line = get_first_line(reversed(self.indented))
            
            # The current indent level so far.
            prior_indent = get_line_indent(last_line)

            # Does the last line enter a block statement
            definition = last_line.rstrip().endswith(':')
            returns = _has_return(self.indented)

            this_indent = get_line_indent(get_first_line(self.raw))
            
            # Assign the minimum indent 
            if not self.min_indent: 
                self.min_indent = this_indent
            
            if this_indent < self.min_indent:
                code = textwrap.indent(code, ' '*(self.min_indent-this_indent))
                this_indent = get_line_indent(get_first_line(code.splitlines()))

                
            # Normalize the indent we'll assign the body+code
            indent = max(self.min_indent, (returns and min or max)(prior_indent, this_indent))        
            
            if definition:
                if prior_indent >= indent:
                    indent = (prior_indent + 4)
                
                body = hanging_indent(textwrap.indent(body, ' '*self.min_indent), ' '*(indent-self.min_indent))
            else:
                body = textwrap.indent(body, ' '*indent)
                
            # Cell Magics
            if code.lstrip().startswith('%%'):
                # Cell magics can be split across __Markdown__ blocks.  With this 
                # approach conditional blocks can be used with magics.
                code = (' '*this_indent) + importnb.loader.dedent(code)
                # might have to add lines if line sized changed.
                
            if self.min_indent:
                self.indented.extend(body.splitlines(True) + code.splitlines(True))
                self.unindented = []
            self.raw = []
            return ''       

        def parse_def_footnotes(self, m):
            self.format()
            text = ''.join(self.pop_text(m))
            key, sep, body = text.lstrip('[').lstrip('^').partition(']:')
            key = quote(key)
            quoted = quote(body.lstrip())
            self.unindented.extend(F"""globals()[{key}] = {quoted if quoted.strip() else "None"}""".splitlines(True))
            
        def parse_def_links(self, m):
            
            self.parse_block_code()
            text = ''.join(self.pop_text(m))
            

            key, sep, body = text.lstrip('[').lstrip('^').partition(']:')
            key = quote(key)

            self.unindented.extend(F"""globals()[{key}] = globals().get({key}, {quote(body.lstrip(), ')')}""".splitlines(True))

    def markdown_to_python(str)->"Valid Python Source": 
        return TanglePidginBlockLexer().parse(str)

    print(markdown_to_python("""### [The `__test__` variable][which docstrings are examined]

    `__test__` is a special `object` that `doctest.DocTestFinder` recognizes. It is a `dict` that with values that have tests.

        __test__ = {}

    [which docstrings are examined]: https://docs.python.org/2/library/doctest.html#which-docstrings-are-examined"""))

print(markdown_to_python("""   
### An `nbconvert.preprocessors.Preprocessor`

        def preprocess_cell(self, cell, resources, index):
            if 'source' not in cell: return cell, resources
            
            if cell['cell_type'] == 'code':
                self.is_pidgin = self.is_pidgin or (
                    'load_ext pidgin' in ''.join(cell['source']))
               
            if cell['cell_type'] in (['markdown'] + (
                self.is_pidgin and ['code'] or []
            )):
                if (cell['cell_type'] == 'markdown') or ('outputs' in cell) and cell['outputs']:
                    cell = add_slide_data(cell)
                
            return cell, resources
        
"""))

print(markdown_to_python("""    class AutomaticallyPresent(nbconvert.preprocessors.Preprocessor):
### An `nbconvert.preprocessors.Preprocessor`

        def preprocess_cell(self, cell, resources, index):
            if 'source' not in cell: return cell, resources
            
            if cell['cell_type'] == 'code':
                self.is_pidgin = self.is_pidgin or (
                    'load_ext pidgin' in ''.join(cell['source']))
               
            if cell['cell_type'] in (['markdown'] + (
                self.is_pidgin and ['code'] or []
            )):
                if (cell['cell_type'] == 'markdown') or ('outputs' in cell) and cell['outputs']:
                    cell = add_slide_data(cell)
                
            return cell, resources
        
        def preprocess(self, nb, resources):
            self.last_heading = None
            self.is_pidgin = False
            nb, resources = super().preprocess(nb, resources)
            return nbconvert.exporters.slides.prepare(nb), resources
"""))

print(    markdown_to_python("""    
### What you will learn

* How interactive notebook computing is similar to manual testing.
* How to use the __builtin__ Python [^🐍] testing tools `doctest` and `unittest` in the notebooks

            if False:
[^🐍]: llll
"""))

print(    markdown_to_python("""    class AutomaticallyPresent(nbconvert.preprocessors.Preprocessor):
### An `nbconvert.preprocessors.Preprocessor`
    
    
>>> assert nbconvert.SlidesExporter(preprocessors=[AutomaticallyPresent()]).from_filename(
...    globals().get('__file__', 'Untitled104.ipynb'))[0]

        def preprocess_cell(self, cell, resources, index):
            if 'source' not in cell: return cell, resources
            
            if cell['cell_type'] == 'code':
                self.is_pidgin = self.is_pidgin or (
                    'load_ext pidgin' in ''.join(cell['source']))
               
            if cell['cell_type'] in (['markdown'] + (
                self.is_pidgin and ['code'] or []
            )):
                if (cell['cell_type'] == 'markdown') or ('outputs' in cell) and cell['outputs']:
                    cell = add_slide_data(cell)
                
            return cell, resources
        
        def preprocess(self, nb, resources):
            self.last_heading = None
            self.is_pidgin = False
            nb, resources = super().preprocess(nb, resources)
            return nbconvert.exporters.slides.prepare(nb), resources
"""))

`markdown_to_python` converts __Markdown__ to __Python__ in a semi lossless way.

    print(markdown_to_python("""## Converting a notebook to a presentation.

    The `nbconvert.SlidesExporter` exports `nbformat` documents as presentations;
    `assert issubclass(nbconvert.SlidesExporter, nbconvert.HTMLExporter)`.

            import nbconvert, nbformat, traitlets"""))

    print(markdown_to_python("""* AAA
    * BBBBBBB
        1. CCC

        2. DDDD
    
                            ...
    * EEE

    QWEF"""))

    print(markdown_to_python("""## Special `pidgin.inspector` indentifiers.

- __object?__ or __object??__ - one question mark returns basic info, two question marks returns the source with the `object` info.  Question marks supercede all other 
delimiters.


        question_marks = "([\S]+)\?{1}|\?{1}([\S]+)"
- __{{brace[0]}}"expression or statement"{{brace[1]}}__ 
- author's recieve a preview of the `jinja2` expression.


        braced = "(\{){2}([^(\?}{2})]+)\}{2}"
- __{{tick}}"expression or statement"{{tick}}__ - executes the inline code and displays a preview.  _Statements and expressions in inline code
are not rendered by `pidgin`, but intermediate representations assist in authoring `pidgin` documents._


        ticked = "`([^\?]+)`"
    
The conditions for a regular expression used to discover substrings.

        inline_code_objects = re.compile(F"{'|'.join((braced, ticked, question_marks))}")
                                        
When no `inline_code_objects` are found the `pidgin.inspector` defaults to a __Markdown__ preview."""))

    print(markdown_to_python("""This is 
    
            a=\\
    * list
    
    
            h = \\
    * lasdfaf"""))

    print(markdown_to_python("""## Special `pidgin.inspector` indentifiers.

        y = \\
- __object?__ or __object??__ - one question mark returns basic info, two question marks returns the source with the `object` info.  Question marks supercede all other 
delimiters.

        question_marks = "([\S]+)\?{1}|\?{1}([\S]+)"
- __{{brace[0]}}"expression or statement"{{brace[1]}}__ - author's recieve a preview of the `jinja2` expression.

        braced = "(\{){2}([^(\?}{2})]+)\}{2}"
- __{{tick}}"expression or statement"{{tick}}__ - executes the inline code and displays a preview.  _Statements and expressions in inline code
are not rendered by `pidgin`, but intermediate representations assist in authoring `pidgin` documents._

        ticked = "`([^\?]+)`"

The conditions for a regular expression used to discover substrings.

        inline_code_objects = re.compile(F"{'|'.join((braced, ticked, question_marks))}")

When no `inline_code_objects` are found the `pidgin.inspector` defaults to a __Markdown__ preview."""))

print(99,markdown_to_python("""## Using the inspector for welcome and help messaging.

A blank cell is the perfect time to present users with instructions.  `pidgin` uses the `welcome` message describe it's features.

    welcome =\\
# Welcome to the `pidgin` inspector

The `pidgin` inspector makes it easier execute inline code and explore python `object`s.

* __?__ take priority & inspect the current `object`
* The default representation is the cell as __Markdown__ source.

---

    welcome = IPython.display.Markdown(welcome)

__*WIP*__"""))

    markdown_to_python("""testing [^1][a]
    ...
    
    [a]: b
    ---
    
        range
    
    [^1]: __bold__                """)

In [ ]:
    def transformer(lines: "that end with a newline."): 
        # Always add a new line
        return markdown_to_python(''.join(lines + ['\n'])).splitlines(1)

In [ ]:
    def load_ipython_extension(ip=None):
        ip = ip or IPython.get_ipython()
        ip.input_transformer_manager.cleanup_transforms = [transformer] + [
            object for object in ip.input_transformer_manager.cleanup_transforms
            if object not in {transformer, IPython.core.inputtransformer2.classic_prompt}
        ]

    def unload_ipython_extension(ip=None):
        ip = ip or IPython.get_ipython()
        ip.input_transformer_manager.cleanup_transforms = [
            object for object in ip.input_transformer_manager.cleanup_transforms
            if object is not transformer]
    if __name__ == '__main__': load_ipython_extension()